# Everruns Agent API Example

This notebook demonstrates how to use the Everruns API to:
1. Create an agent with a system prompt
2. Create a session (conversation)
3. Send a message to trigger the agentic loop
4. Poll for session completion and retrieve the response
5. Stream real-time events via SSE (optional)

## Prerequisites

- Everruns API server running at `http://localhost:9000`
- Python packages: `requests`, `sseclient-py` (for SSE streaming)

In [ ]:
# Install required packages
!pip install requests sseclient-py

In [ ]:
import requests
import time
import json
from typing import Optional

# Configuration
BASE_URL = "http://localhost:9000"
API_V1 = f"{BASE_URL}/v1"

# Helper functions
def print_json(data):
    """Pretty print JSON data."""
    print(json.dumps(data, indent=2, default=str))

## 1. Health Check

First, verify the API server is running.

In [ ]:
response = requests.get(f"{BASE_URL}/health")
response.raise_for_status()
print("API Status:", response.json())

## 2. Create an Agent

An agent is a configuration for an agentic loop with a system prompt.

In [ ]:
# Create an agent
agent_data = {
    "name": "Python Example Agent",
    "description": "A helpful assistant created via the Python API",
    "system_prompt": "You are a helpful assistant. Be concise and friendly.",
    "tags": ["example", "python"]
}

response = requests.post(f"{API_V1}/agents", json=agent_data)
response.raise_for_status()
agent = response.json()

agent_id = agent["id"]
print(f"Created agent with ID: {agent_id}")
print_json(agent)

## 3. Create a Session

A session is an instance of conversation with the agent.

In [ ]:
# Create a session
session_data = {
    "title": "Python API Test Session",
    "tags": ["test"]
}

response = requests.post(f"{API_V1}/agents/{agent_id}/sessions", json=session_data)
response.raise_for_status()
session = response.json()

session_id = session["id"]
print(f"Created session with ID: {session_id}")
print(f"Session status: {session['status']}")
print_json(session)

## 4. Send a Message

Sending a user message triggers the agentic loop workflow.

In [ ]:
# Send a user message
message_data = {
    "message": {
        "role": "user",
        "content": [
            {"type": "text", "text": "What is 2 + 2? Please explain your reasoning."}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()
user_message = response.json()

print(f"Created message with ID: {user_message['id']}")
print_json(user_message)

## 5. Wait for Completion

Poll the session status until it returns to `pending` (ready for more messages).

In [ ]:
def wait_for_session_ready(agent_id: str, session_id: str, timeout: int = 60) -> dict:
    """Poll session status until it's ready for more messages.
    
    Status flow: pending -> running -> pending (cycle continues)
    """
    start_time = time.time()
    prev_status = None
    
    while time.time() - start_time < timeout:
        response = requests.get(f"{API_V1}/agents/{agent_id}/sessions/{session_id}")
        response.raise_for_status()
        session = response.json()
        status = session["status"]
        
        if status != prev_status:
            print(f"Session status: {status}")
            prev_status = status
        
        if status == "pending":
            return session
        elif status == "failed":
            raise Exception(f"Session failed: {session}")
        
        time.sleep(0.5)
    
    raise TimeoutError(f"Session did not complete within {timeout} seconds")

# Wait for the session to process the message
print("Waiting for agent to respond...")
final_session = wait_for_session_ready(agent_id, session_id)
print("Agent response complete!")

## 6. Get All Messages

Retrieve the full conversation including the assistant's response.

In [ ]:
# List all messages in the session
response = requests.get(f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages")
response.raise_for_status()
messages_response = response.json()

print(f"Total messages: {messages_response['total']}\n")

for msg in messages_response["items"]:
    role = msg["role"]
    print(f"--- {role.upper()} ---")
    
    for part in msg["content"]:
        if part["type"] == "text":
            print(part["text"])
        elif part["type"] == "tool_call":
            print(f"[Tool Call: {part['name']}({part['arguments']})]")
        elif part["type"] == "tool_result":
            print(f"[Tool Result: {part['result']}]")
    print()

## 7. Continue the Conversation

Send another message to continue the conversation.

In [ ]:
# Send a follow-up message
followup_data = {
    "message": {
        "content": [
            {"type": "text", "text": "What about 3 + 3?"}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=followup_data
)
response.raise_for_status()

# Wait for response
print("Waiting for agent to respond...")
wait_for_session_ready(agent_id, session_id)
print("Done!\n")

# Get updated messages
response = requests.get(f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages")
response.raise_for_status()
messages = response.json()["items"]

# Show the last two messages (user question and assistant answer)
for msg in messages[-2:]:
    role = msg["role"]
    print(f"--- {role.upper()} ---")
    for part in msg["content"]:
        if part["type"] == "text":
            print(part["text"])
    print()

## 8. Real-time Events with SSE (Optional)

Stream real-time events from the session using Server-Sent Events.

In [ ]:
import threading
import sseclient

def stream_events(agent_id: str, session_id: str, stop_event: threading.Event):
    """Stream SSE events from a session."""
    url = f"{API_V1}/agents/{agent_id}/sessions/{session_id}/events"
    
    try:
        response = requests.get(url, stream=True, headers={"Accept": "text/event-stream"})
        client = sseclient.SSEClient(response)
        
        for event in client.events():
            if stop_event.is_set():
                break
            
            print(f"Event: {event.event}")
            if event.data:
                try:
                    data = json.loads(event.data)
                    print_json(data)
                except json.JSONDecodeError:
                    print(f"Data: {event.data}")
            print()
    except Exception as e:
        if not stop_event.is_set():
            print(f"SSE error: {e}")

# Start SSE streaming in background
stop_event = threading.Event()
sse_thread = threading.Thread(target=stream_events, args=(agent_id, session_id, stop_event))
sse_thread.daemon = True
sse_thread.start()

print("SSE streaming started. Send a message to see events...")

In [ ]:
# Send a message while SSE is streaming
message_data = {
    "message": {
        "content": [
            {"type": "text", "text": "Tell me a short joke."}
        ]
    }
}

response = requests.post(
    f"{API_V1}/agents/{agent_id}/sessions/{session_id}/messages",
    json=message_data
)
response.raise_for_status()

# Wait a moment for events to stream
time.sleep(5)

# Wait for completion
wait_for_session_ready(agent_id, session_id)
print("Message processed!")

In [ ]:
# Stop SSE streaming
stop_event.set()
print("SSE streaming stopped.")

## 9. Cleanup

Delete the session and archive the agent.

In [ ]:
# Delete the session
response = requests.delete(f"{API_V1}/agents/{agent_id}/sessions/{session_id}")
if response.status_code == 204:
    print(f"Session {session_id} deleted.")
else:
    print(f"Failed to delete session: {response.status_code}")

# Archive the agent
response = requests.delete(f"{API_V1}/agents/{agent_id}")
if response.status_code == 204:
    print(f"Agent {agent_id} archived.")
else:
    print(f"Failed to archive agent: {response.status_code}")

## Summary

This notebook demonstrated the core Everruns API workflow:

1. **Create Agent** - `POST /v1/agents` - Define an agent with a system prompt
2. **Create Session** - `POST /v1/agents/{id}/sessions` - Start a conversation
3. **Send Message** - `POST /v1/agents/{id}/sessions/{id}/messages` - Trigger the agentic loop
4. **Poll Status** - `GET /v1/agents/{id}/sessions/{id}` - Wait for `pending` status
5. **Get Messages** - `GET /v1/agents/{id}/sessions/{id}/messages` - Retrieve conversation
6. **Stream Events** - `GET /v1/agents/{id}/sessions/{id}/events` - Real-time updates (SSE)

For more details, see the [API documentation](http://localhost:9000/swagger-ui/).